# Install necessary libraries for Kalman Filter

In [1]:
!git clone https://github.com/niconielsen32/ObjectTracking.git
!pip install -r /content/ObjectTracking/requirements.txt

Cloning into 'ObjectTracking'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 43 (delta 1), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), 10.84 MiB | 6.64 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=dd0489d0547b01ae9afbbd4b0a41ceb750cdb5332c4634ef8ab35456895a4cd1
  Stored in directory: /root/.cache/pip/wheels/fe/f6/cb/40331472edf4fd399b8cad02973c6acbdf26898342928327fe
  Created wheel for lap:

Import libraries

In [2]:
import sys
import os
from google.colab import files

sys.path.append("/content/ObjectTracking")
from sort import Sort
from lib import VisTrack, show_video, create_video

import numpy as np
import PIL.Image
import cv2
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import shutil
from google.colab.patches import cv2_imshow

### Helper Functions
These functions are used to remove extra boxes from the detections

In [3]:
def nms(bounding_boxes, threshold):
    # If no bounding boxes, return empty list
    if len(bounding_boxes) == 0:
        return [], []

    # Bounding boxes
    boxes = np.array(bounding_boxes)

    # coordinates of bounding boxes
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]

    # Confidence scores of bounding boxes

    # Picked bounding boxes
    picked_boxes = []

    # Compute areas of bounding boxes
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)

    # Sort by confidence score of bounding boxes
    order = np.argsort(areas)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]

        # Pick the bounding box with largest confidence score
        picked_boxes.append(bounding_boxes[index])

        # Compute ordinates of intersection-over-union(IOU)
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])

        # Compute areas of intersection-over-union
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h

        # Compute the ratio between intersection and union
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)

        left = np.where(ratio < threshold)
        order = order[left]

    return picked_boxes

def remove_bb_rot(boxes, rot, threshold=0.5):
    # Bounding boxes
    boxes = np.array(boxes)

    # coordinates of bounding boxes
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]

    # Compute areas of bounding boxes
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)

    rot_x ,rot_y, rot_x1, rot_y1 = rot
    rot_area = (rot_x1 - rot_x + 1) * (rot_y1 - rot_y + 1)

    x1 = np.maximum(rot_x, start_x)
    x2 = np.minimum(rot_x1, end_x)
    y1 = np.maximum(rot_y, start_y)
    y2 = np.minimum(rot_y1, end_y)

    # Compute areas of intersection-over-union
    w = np.maximum(0.0, x2 - x1 + 1)
    h = np.maximum(0.0, y2 - y1 + 1)
    intersection = w * h

    # Compute the ratio between intersection and union
    ratio = intersection / areas

    left = np.where(ratio > threshold)

    # print(np.sum(left))
    if np.sum(left)>0:
      return boxes[left]
    else:
      return boxes

In [26]:
# set necessary variables

# Which video do you want to use?
filename = 'video_cut.mp4'#@param ["video_input8.mp4", "video_cut.mp4"]

# Specified RoI for both the videos
if filename == "video_cut.mp4":
  rot = [360, 100, 1580, 860]
  # Player 1
  p1 = [1055, 233]
  # Player 2 
  p2 = [506, 825]
  hits1, hits2 = 1, 0
elif filename == "video_input8.mp4":
  rot = [350, 110, 1580, 840]
  p1 = [889, 117]
  p2 = [567, 850]
  bg1 = [1127, 127]
  bg2 = [765, 121]
  hits1, hits2 = 0, 0


draw_imgs = []

# Variables indicating the tracking IDs of players and ball
player1_id = -1
player2_id = -1
ball_id = -1
# Variable indicating the direction of vertical movement
ball_dir = 1
ball_prev = [960, 390]
# current location of players
player1_prev = [-1,-1]
player2_prev = [-1,-1]
# Used for calculating scores
p1_score= False
p2_score= False

# video 2
pt0 = [726,76]
pt1 = [292,963]
pt2 = [1172,76]
pt3 = [1634, 953]

# Variables to estimate tracking loss
ball_tracked = 0
p1_tracked = 0
p2_tracked = 0

## Detect and Track using Kalman Filter

In [27]:
# Define Kalman Filter
sort = Sort(max_age=15, min_hits=10, iou_threshold=0.03)

# Define bg subtractor
BG_SUB = cv2.createBackgroundSubtractorKNN()

# Define opencv object to read the video
vidcap = cv2.VideoCapture(cv2.samples.findFile(filename))

# Get video properties
length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(vidcap.get(3))
frame_height = int(vidcap.get(4))
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
size = (frame_width, frame_height)
result = cv2.VideoWriter(f'mytrackresult_{filename.split(".")[0].split("_")[1]}_withdir.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         fps, size)

# Define RoI
region_of_interest = np.zeros((frame_height, frame_width, 3))
points = np.array([pt0, pt1, pt3, pt2])
cv2.fillPoly(region_of_interest, pts=[points], color=(255, 255, 255))
region_of_interest = cv2.cvtColor(region_of_interest.astype('uint8'), cv2.COLOR_BGR2GRAY) 
ret,region_of_interest = cv2.threshold(region_of_interest,250,255,0)

# Progress bar for aesthetic displays
pbar = tqdm(total=length-4)
i = 0
while True:
    ret, frame = vidcap.read()
    if not ret:
        break
    # apply BG_SUB to each frame
    fg_mask = BG_SUB.apply(frame)
    # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
    fg_mask = cv2.erode(fg_mask.astype('uint8')*255, np.ones((3, 3)))
    fg_mask = cv2.dilate(fg_mask.astype('uint8')*255, np.ones((5, 5)))
    fg_mask = cv2.bitwise_and(fg_mask.astype('uint8'), region_of_interest.astype('uint8'))
    contours, hierarchy = cv2.findContours(image=fg_mask, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    rgb_img = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)
    contours = [c for c in contours if cv2.contourArea(c)<30000]
    contours = sorted(contours, key=lambda x: cv2.contourArea(x))
    i+=1
    if i < 5:
      continue
    boxes = []
    for c in contours:
          (x, y, w, h) = cv2.boundingRect(c)
          boxes.append([x-15,y-15, x+w+15,y+h+15])

    #NMS
    boxes = nms(boxes, threshold=0.01)
    if len(boxes) != 0:
       if boxes[0] != []:
          boxes = remove_bb_rot(boxes, rot, threshold=0.4)  

    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
      
    # fontScale
    fontScale = 0.7
      
    # Blue color in BGR
    color = (255, 0, 0)
      
    # Line thickness of 2 px
    thickness = 2
    detections_in_frame = len(boxes)
    if detections_in_frame:
        scores = [1]*len(boxes)
        classes = [1]*len(boxes)
    else:
        boxes = np.empty((0, 5))
    dets = np.hstack((boxes, np.array(scores).reshape((len(boxes),1))))
    res = sort.update(dets)
    # print(res)
    boxes_track = res[:,:-1]
    boxes_ids = res[:,-1].astype(int)
    cv2.rectangle(frame, (1200,800), (1870,950), (0, 0, 0), 2)
    cv2.rectangle(frame, (1200,800), (1870,950), (255, 255, 255), -1)
    if detections_in_frame:
        for j , box in enumerate(boxes_track):
          (x, y, x1, y1)= box
          (x, y, x1, y1) = (int(x), int(y), int(x1), int(y1))
          area = abs((x1-x)*(y1-y))
          if p1[0]>x and p1[0]<x1 and p1[1]>y and p1[1]<y1 and i == 5:
            player1_id = boxes_ids[j]
          elif p2[0]>x and p2[0]<x1 and p2[1]>y and p2[1]<y1 and i == 5:
            player2_id = boxes_ids[j]
          elif area >5000 and y<400:
            if player1_id not in boxes_ids:
              player1_id = boxes_ids[j]
          elif area >5000 and y>600:
            if player2_id not in boxes_ids:
              player2_id = boxes_ids[j]

        for j , box in enumerate(boxes_track):
          (x, y, x1, y1)= box
          (x, y, x1, y1) = (int(x), int(y), int(x1), int(y1))
          area = abs((x1-x)*(y1-y))

          if player1_id in boxes_ids and player2_id in boxes_ids:
            if area<3000 and area>200 and \
                (y+y1)/2 > boxes_track[np.where(boxes_ids == player1_id)[0]][0][3] and \
                (y+y1)/2 < boxes_track[np.where(boxes_ids == player2_id)[0]][0][1]:
                if filename == "video_input8.mp4":
                  if (bg1[0]>x and bg1[0]<x1 and bg1[1]>y and bg1[1]<y1) or (bg2[0]>x and bg2[0]<x1 and bg2[1]>y and bg2[1]<y1):
                    continue
                if ball_id not in boxes_ids:
                  ball_id = boxes_ids[j]
          else:
            if area<3000 and area>200:
                if filename == "video_input8.mp4":
                  if (bg1[0]>x and bg1[0]<x1 and bg1[1]>y and bg1[1]<y1) or (bg2[0]>x and bg2[0]<x1 and bg2[1]>y and bg2[1]<y1):
                    continue
                if ball_id not in boxes_ids:
                  ball_id = boxes_ids[j]

          # Using cv2.putText() method
          if boxes_ids[j] == player1_id:
            cv2.rectangle(frame, (x,y), (x1,y1), (0, 255, 100), 2)
            cv2.rectangle(frame, (x,y-30), (x1,y), (0, 255, 100), -1)
            frame = cv2.putText(frame, "Player 1", (x,y-10), font, 
                   fontScale, (255,255,255), thickness, cv2.LINE_AA)
            player1_prev = [x,y]
            p1_tracked += 1
          elif boxes_ids[j] == player2_id:
            cv2.rectangle(frame, (x,y), (x1,y1), (0, 0, 255), 2)
            cv2.rectangle(frame, (x,y-30), (x1,y), (0, 0, 255), -1)
            frame = cv2.putText(frame, "Player 2", (x,y-10), font, 
                   fontScale, (255,255,255), thickness, cv2.LINE_AA)
            player2_prev = [x,y]
            p2_tracked += 1
          elif boxes_ids[j] == ball_id:
            ball_tracked += 1
            cv2.rectangle(frame, (x,y), (x1,y1), (255, 0, 255), 2)
            cv2.rectangle(frame, (x,y-30), (x1,y), (255, 0, 255), -1)
            frame = cv2.putText(frame, "Ball", (x,y-10), font, 
                   0.7, (255,255,255), thickness, cv2.LINE_AA)
            if y> ball_prev[1] and abs( y- ball_prev[1])>40:
              if ball_dir == -1:
                hits1 += 1
              ball_dir = 1
              ball_prev[1] = y
              ball_prev[0] = x
            if y< ball_prev[1] and abs( y- ball_prev[1])>40:
              if ball_dir == 1:
                hits2 += 1
              ball_dir = -1
              ball_prev[1] = y
              ball_prev[0] = x
            if i> 0.85*length and x<0.4*frame_width and y<0.2*frame_height:
              print("Player 2 scores")
              p2_score = True
            elif i> 0.85*length and x>0.4*frame_width and y>0.4*frame_height:
              print("Player 1 scores")
              p1_score = True
    frame = cv2.putText(frame, "Player 1: "+ str(hits1)+"  Player 2: "+ str(hits2), (1250,850), font, 
                   1, (0,0,0), thickness, cv2.LINE_AA)
    if p1_score:
        frame = cv2.putText(frame, "Point to Player 1", (1250,900), font, 
                   1, (0, 255, 100), thickness, cv2.LINE_AA)
    if p2_score:
        frame = cv2.putText(frame, "Point to Player 2", (1250,900), font, 
                   1, (0,0,255), thickness, cv2.LINE_AA)
    result.write(frame.astype('uint8'))
    pbar.update(1)

  0%|          | 0/925 [00:00<?, ?it/s]

Player 1 scores
Player 1 scores
Player 1 scores
Player 1 scores
Player 1 scores
Player 1 scores
Player 1 scores
Player 1 scores
Player 1 scores


In [28]:
print("The ball was tracked in {} frames, \nPlayer 1 was tracked in {} frames\nPlayer 2 was tracked in {} frames".format(ball_tracked, p1_tracked, p2_tracked))

The ball was tracked in 435 frames, 
Player 1 was tracked in 911 frames
Player 2 was tracked in 925 frames


In [25]:
print("The ball was tracked in {} frames, \nPlayer 1 was tracked in {} frames\nPlayer 2 was tracked in {} frames".format(ball_tracked, p1_tracked, p2_tracked))

The ball was tracked in 63 frames, 
Player 1 was tracked in 164 frames
Player 2 was tracked in 138 frames
